In [86]:
from ex2_utils import create_epanechnik_kernel, generate_responses_1, get_patch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [99]:
def mean_shift(frame, size, position, eps=0.05):
    # Round to odd number
    kernel_size_x = int(math.ceil(size[0] / 2)) * 2 - 1
    kernel_size_y = int(math.ceil(size[1] / 2)) * 2 - 1

    # Caculate the derivatives of size of kernel
    x = np.arange(-(kernel_size_x // 2), (kernel_size_x // 2) + 1)
    y = np.arange(-(kernel_size_y // 2), (kernel_size_y // 2) + 1)
    xi_X, xi_Y = np.meshgrid(x, y)

    positions = []
    iter_count = 0
    while True:
        wi, _ = get_patch(frame, position, (kernel_size_x, kernel_size_y))

        # Calculate the changes in both x and y directions
        xk_X = np.sum(xi_X * wi) / np.sum(wi)
        xk_Y = np.sum(xi_Y * wi) / np.sum(wi)

        # Update the position accordingly
        position = (position[0] + xk_X, position[1] + xk_Y)
        positions.append(tuple(map(int, np.floor(position))))

        # Check if the algorithm converged
        if abs(xk_X) < eps and abs(xk_Y) < eps:
            break

        # Sanity check if the algorihtm does not converge
        if iter_count >= 500:
            break

        iter_count += 1

    position = tuple(map(int, np.floor(position)))
    return position, iter_count, positions


In [100]:
responses = generate_responses_1()
size = (6, 5)
eps = 0.01
starting_position = (60, 60)
position, iters, positions = mean_shift(responses, size, starting_position, eps)
print(position, iters)

(51, 70) 211
